Notebook based on original code and data from Gutierrez et al. 2020 https://doi.org/10.1038/s41467-019-13867-y.

Load libraries

In [1]:
import cobra
import pdb
import cobra_io_mat # modified cobrapy functions to preserve more model fields when exporting to matlab format

Read files necessary to expand recon2.2

In [2]:
f = open("rxnFormula_HUMAN.txt")
rxnFormula = f.read().splitlines()
f.close()

f = open("rxnAbbreviation_HUMAN.txt")
rxnAbbreviation = f.read().splitlines()
f.close()

f = open("rxnPathway_HUMAN.txt")
rxnPathway = f.read().splitlines()
f.close()

f = open("rxnConditions_HUMAN.txt")
rxnConditions = f.read().splitlines()
f.close()

f = open("rxnGPRs_HUMAN_HGNC.txt")
rxnGPR = f.read().splitlines()
f.close()

# Load PSIM matrix
f = open('PSIM_HUMAN.tab','r')
PSIM=f.read().splitlines()
f.close()

# Extract the Uniprot IDs
PSIM_entries = []
for line in PSIM:
    PSIM_entries.append(line.split('\t')[0])

Define Basic Functions

In [3]:
#Define a functions that counts of amino acids
def count_AAs(sequence):
    AAcounts = []    
    AAs = ['G', 'A', 'V', 'L', 'I', 'M', 'W', 'F', 'P', 'S', 'T', 'C', 'Y', 'N', 'Q', 'E', 'D', 'K', 'R', 'H']
    for aa in AAs:
        AAcounts.append(sequence.count(aa))
    return AAcounts

#Define a function that substitutes question marks for AA counts in a formula (string)
#Useful for translation and protein degradation pathway

def substitute_AAs_count(formula,AAcounts):
    template = "? gly[c] + ? ala_L[c] + ? val_L[c] + ? leu_L[c] + ? ile_L[c] + ? met_L[c] + ? trp_L[c] + ? phe_L[c] + ? pro_L[c] + ? ser_L[c] + ? thr_L[c] + ? cys_L[c] + ? tyr_L[c] + ? asn_L[c] + ? gln_L[c] + ? glu_L[c] + ? asp_L[c] + ? lys_L[c] + ? arg_L[c] + ? his_L[c]"
    new = ''
    index = 0
    for character in template:
        if character == "?":
            new = new + str(AAcounts[index])
            index = index + 1
        else:
            new = new + character
    newFormula = formula.replace(template,new)
    return newFormula
    
# Function that generates the translation reaction of a protein given its UniProt ID
def translate_protein(entryID):
    #Obtain protein sequence and length
    PSI_row = PSIM[PSIM_entries.index(str(entryID))] 
    PSI_row = PSI_row.split('\t')
    sequence = PSI_row[11]
    AAcounts = count_AAs(sequence)
    N = len(sequence) #Number to replace atp's, gtp's, ppi's, pi's, h2o's, amp's, and gpd's
    templateFormula = "? h2o[c] + ? atp[c] + ? gtp[c] + ? gly[c] + ? ala_L[c] + ? val_L[c] + ? leu_L[c] + ? ile_L[c] + ? met_L[c] + ? trp_L[c] + ? phe_L[c] + ? pro_L[c] + ? ser_L[c] + ? thr_L[c] + ? cys_L[c] + ? tyr_L[c] + ? asn_L[c] + ? gln_L[c] + ? glu_L[c] + ? asp_L[c] + ? lys_L[c] + ? arg_L[c] + ? his_L[c] -> ? h[c] + ? amp[c] + adp[c] + ? pi[c] + ? gdp[c] + ? ppi[c] + XXX[c]"
    translationFormula = substitute_AAs_count(templateFormula, AAcounts)
    translationFormula = translationFormula.replace("? h2o[c]",str(2*N-1)+" h2o[c]")
    translationFormula = translationFormula.replace("? h[c]",str(2*N-1)+" h[c]")
    translationFormula = translationFormula.replace("? ppi[c]",str(N)+" ppi[c]")
    translationFormula = translationFormula.replace("? pi[c]",str(2*N-1)+" pi[c]")
    translationFormula = translationFormula.replace("? gdp[c]",str(2*N-2)+" gdp[c]")
    translationFormula = translationFormula.replace("? amp[c]",str(N)+" amp[c]")
    translationFormula = translationFormula.replace("? gtp[c]",str(2*N-2)+" gtp[c]")
    translationFormula = translationFormula.replace("? atp[c]",str(N+1)+" atp[c]")
    translationFormula = translationFormula.replace("XXX[c]",str(entryID)+"[c]")
    return translationFormula

# Function that replaces the XXX template name for the UniProt ID
def insert_prot_name_in_rxnFormula(formula,entryID):
    newFormula = formula.replace("XXX",str(entryID))
    return newFormula

# Function that replaces the XXX template name for the Reaction abbreviation
def insert_prot_name_in_rxnName(rxnAbbrev,entryID):
    newAbbreviation = str(entryID) + "_" + rxnAbbrev
    return newAbbreviation

#Function for adding the reactions of a given PathwayName
def addPathway(pathwayName,listOfRxns,listOfRxnsNames,listOfPathways):   
    newList = listOfRxns    
    newList2 = listOfRxnsNames
    newList3 = listOfPathways
    for i in range(len(rxnPathway)):
        if rxnPathway[i] == pathwayName:
            newList.append(rxnFormula[i])
            newList2.append(rxnAbbreviation[i])
            newList3.append(pathwayName)
    return newList,newList2,newList3

#Function for adding the reactions of a given PathwayName given a condition
def addPathwayFromCondition(conditionName,listOfRxns,listOfRxnsNames,listOfPathways):   
    newList = listOfRxns 
    newList2 = listOfRxnsNames
    newList3 = listOfPathways
    for i in range(len(rxnConditions)):
        if rxnConditions[i] == conditionName:
            newList.append(rxnFormula[i])
            newList2.append(rxnAbbreviation[i])
            newList3.append(conditionName)
    return newList,newList2,newList3

#Function for creating a list of GPRs given a list of Rxn names
def getGPRsFromRxnNames(listOfRxnsNames):
    GPR_list = []
    for reaction in listOfRxnsNames:
        if reaction in rxnAbbreviation:
            GPR_list.append(rxnGPR[rxnAbbreviation.index(reaction)])
        else:
            GPR_list.append('')
    return GPR_list
     
#Function that adds canonical reactions to overall list
def addCanonicalRxns(listOfRxns,listOfRxnNames,listOfGPRs,listOfPathways):
    newListRxns = listOfRxns
    newListNames = listOfRxnNames
    newListGPRs = listOfGPRs
    newListPathways = listOfPathways
    #Add  canonical reactions
    [newListRxns, newListNames, newListPathways] = addPathway("Canonical",listOfRxns,listOfRxnNames,listOfPathways)
    #Add canonical GPRs
    r = []
    n = []
    p = []
    [r,n,p] = addPathway("Canonical",r,n,p)
    newGPRs = getGPRsFromRxnNames(n)
    for gpr in newGPRs:
        newListGPRs.append(gpr)
    return newListRxns, newListNames, newListGPRs, newListPathways


Function that generates protein-specific reactions list given a string of a Uniprot ID

In [4]:
def generateProteinSpecificRxns_A_modified(entryID):
    #Obtain protein sequence and length
    PSI_row = PSIM[PSIM_entries.index(str(entryID))] 
    PSI_row = PSI_row.split('\t')
    sequence = PSI_row[11]
    L = float(PSI_row[2]) # Protein Length
    MW = float(PSI_row[3]) # Molecular weight
    PSI = []
    Kv = 0.7
    V = MW * 1.21 / 1000.0 # Protein Volume in nm^3
    clathrin_coeff = int(round(29880.01 * Kv / V)) # Number of proteins per clathrin vesicle  
    copi_coeff = int(round(143793.19 * Kv / V))
    copii_coeff = int(round(268082.35 * Kv / V))
    connector = ''
    #Prepare vectors that will store reactions and components
    protName = str(entryID)
    rxns = []
    rxnNames = []
    subSystems = []

    #Add translation reaction
    translation_reaction = translate_protein(protName)
    rxns.append(translation_reaction)
    rxnNames.append("TRANSLATION_protein")
    subSystems.append("Translation")
    for i in [0,4,5,6,7,8,9,10]:
        PSI.append(PSI_row[i])  #This is the vector from the PSIM that corresponds to the given protein 
										  #[entry,SP,DSB,GPI,NG,OG,TMD,SubCellLoc] 

#------------------------------------------------------------------------------ 
    #Add translocation reactions PSI[1] = "Signal Peptide"
    
    if PSI[1] == '0': #If it doesn't have signal peptide then ignore
        GPRs = getGPRsFromRxnNames(rxnNames)
        #Change the reaction names and their formulas to include the UniProt Identifier
        for i in range(len(rxns)):
            rxns[i] = insert_prot_name_in_rxnFormula(rxns[i],protName)
        for i in range(len(rxnNames)):
            rxnNames[i] = insert_prot_name_in_rxnName(rxnNames[i],protName)
        rxns.append(protName + '[c] -> ')
        rxnNames.append(protName + '_Final_demand')
        GPRs.append('')
        subSystems.append('Exchange/demand reaction')
        return rxns,rxnNames,GPRs,subSystems
        
    elif PSI[1] == '1': #Translocate protein if it has signal peptide
        if L <= 160:
            [rxns,rxnNames,subSystems] = addPathway("Post-translational Translocation",rxns,rxnNames,subSystems)
        if PSI[7] == '[e]' or PSI[7] == '':
            [rxns,rxnNames,subSystems] = addPathway("Post-translational Translocation (Secretory protein)",rxns,rxnNames,subSystems)
        if PSI[7] == "[pm]" or PSI[6] != '0':
            [rxns,rxnNames,subSystems] = addPathway("Post-translational Translocation (Tail anchored membrane protein)",rxns,rxnNames,subSystems)
        else:
            [rxns,rxnNames,subSystems] = addPathway("Translocation",rxns,rxnNames,subSystems)
        
        number_BiP = L/40 #Number of BiPs depends on protein length http://www.cshperspectives.com/content/5/5/a013201.full
        for i in range(len(rxns)):
            rxns[i] = rxns[i].replace("!",str(number_BiP))
        connector = 'XXX[r]'
        
#------------------------------------------------------------------------------
    #Add Disulphide Bond Reactions
    if PSI[2] != '0':
        number_DSB = PSI[2]
        DSBrxns = []
        DSBrxnNames = []
        DSBsubSystems = []
        DSBrxns.append(connector + ' -> XXX_preDSB[r]')
        DSBrxnNames.append('Start_DSB')
        DSBsubSystems.append('DSB>0')
        [DSBrxns, DSBrxnNames, DSBsubSystems] = addPathwayFromCondition('DSB>0',DSBrxns,DSBrxnNames,DSBsubSystems)
        for i in range(len(DSBrxns)):
            if '?' in DSBrxns[i]:
                if number_DSB == '1':
                    DSBrxns[i] = DSBrxns[i].replace('? ','')
                else:
                    DSBrxns[i] = DSBrxns[i].replace('?',number_DSB)
        
        connector = 'XXX_DSB[r]'

        for reaction in DSBrxns:
            rxns.append(reaction)
        for reactionName in DSBrxnNames:
            rxnNames.append(reactionName)
        for pathwayName in DSBsubSystems:
            subSystems.append(pathwayName)
#------------------------------------------------------------------------------ 
    #Add GPI reactions
    if PSI[3] == '1':
        rxns.append(connector + ' -> XXX_preGPI[r]')
        rxnNames.append('Start_GPI')
        subSystems.append('GPI=1')
        [rxns,rxnNames,subSystems] = addPathwayFromCondition('GPI=1',rxns,rxnNames,subSystems)
        connector = 'XXX-dgpi_hs[r]'
#------------------------------------------------------------------------------    
    #Add N-glycosylation reactions
    if PSI[4] != '0': #Has N-glycans?
        rxns.append(connector + ' -> XXX_preNG[r]')
        rxnNames.append('Start_NG')
        subSystems.append('Golgi processing N')
        number_Nglycans = PSI[4] #Get number of N-Glycans
        NGlyrxns = []
        NGlyrxnNames = []
        NGlysubSystems = []
        
        [NGlyrxns,NGlyrxnNames,NGlysubSystems] = addPathwayFromCondition('NG>0',NGlyrxns,NGlyrxnNames,NGlysubSystems)
        [NGlyrxns,NGlyrxnNames,NGlysubSystems] = addPathway('Golgi processing N',NGlyrxns,NGlyrxnNames,NGlysubSystems)
        for i in range(len(NGlyrxns)): #Change the '?' for the number of N-glycans
            if NGlyrxns[i] == 'XXX-M5-unfold-UBIQP[c] + ? h2o[c] + RAD23A[c] =>  XXX-unfold-UBIQP-RAD23A[c] + ? acgam[c] + ? man[c]':
                h2o = str(7*int(number_Nglycans))
                acgam = str(2*int(number_Nglycans))
                man = str(5*int(number_Nglycans))
                NGlyrxns[i] = 'XXX-M5-unfold-UBIQP[c] + ' + h2o +' h2o[c] + RAD23A[c] =>  XXX-unfold-UBIQP[c]-RAD23A[c] + ' + acgam + ' acgam[c] + ' + man +' man[c]'
            if '?' in NGlyrxns[i]:
                    if number_Nglycans == '1':
                        NGlyrxns[i] = NGlyrxns[i].replace('? ', '')
                    else:
                        NGlyrxns[i] = NGlyrxns[i].replace('?', number_Nglycans)

        for reaction in NGlyrxns:
            rxns.append(reaction)
        for reactionName in NGlyrxnNames:
            rxnNames.append(reactionName)
        for pathwayName in NGlysubSystems:
            subSystems.append(pathwayName)
        
        connector = 'XXX-M8B[r]'
           
#------------------------------------------------------------------------------
    #Add  COPII reactions
    if connector == 'XXX-M8B[r]':
        copii_rxns = []
        copii_names = []
        copii_systems = []
        [copii_rxns, copii_names, copii_systems] =  addPathway('COPII_NG',copii_rxns,copii_names,copii_systems)       
        for i in range(len(copii_rxns)):
            copii_rxns[i] = copii_rxns[i].replace("!",str(copii_coeff))
            rxns.append(copii_rxns[i])
            rxnNames.append(copii_names[i])
            subSystems.append(copii_systems[i])
        
        #[rxns,rxnNames] = addPathway('COPII_NG',rxns,rxnNames)
        connector = 'XXX-M3-GN2[g]'
        
        #-------------- ADD IF PROTEIN = EPO (HUMAN)--------------------------#
        # IMPORTANT! -> Reactions in this pathway assume EPO has NG=3 and Og=1 (see PSIM)
        if protName == 'P01588':
            rxns.append('P01588[c] -> EPO_Human[c]')
            rxnNames.append('make_EPO')
            subSystems.append('Golgi processing (EPO specific)')
            [rxns,rxnNames,subSystems] = addPathway('Golgi processing (EPO specific)',rxns,rxnNames,subSystems)
            connector = 'XXX-M3-GN4-GL4-NA4-F[g]'
            protName = 'EPO_Human'
        
    elif connector == 'XXX-dgpi_hs[r]':
        copii_rxns = []
        copii_names = []
        copii_systems = []
        [copii_rxns, copii_names, copii_systems] =  addPathway('COPII_GPI',copii_rxns,copii_names,copii_systems)       
        for i in range(len(copii_rxns)):
            copii_rxns[i] = copii_rxns[i].replace("!",str(copii_coeff))
            rxns.append(copii_rxns[i])
            rxnNames.append(copii_names[i])
            subSystems.append(copii_systems[i])
        
        #[rxns,rxnNames] = addPathway('COPII_GPI',rxns,rxnNames)
        connector = 'XXX-dpgi_hs[g]'
        
    elif connector == 'XXX_DSB[r]':
        [rxns,rxnNames,subSystems] = addPathway('COPII_DSB',rxns,rxnNames,subSystems)
        
        copii_rxns = []
        copii_names = []
        copii_systems = []
        [copii_rxns, copii_names, copii_systems] =  addPathway('COPII-canonical',copii_rxns,copii_names,copii_systems)       
        for i in range(len(copii_rxns)):
            copii_rxns[i] = copii_rxns[i].replace("!",str(copii_coeff))
            rxns.append(copii_rxns[i])
            rxnNames.append(copii_names[i])
            subSystems.append(copii_systems[i])
        
        #[rxns,rxnNames] = addPathway('COPII-canonical',rxns,rxnNames)
        connector = 'XXX[g]'
    
    elif connector == 'XXX[r]':
        [rxns,rxnNames,subSystems] = addPathway('COPII-normal',rxns,rxnNames,subSystems)
        
        copii_rxns = []
        copii_names = []
        copii_systems = []
        [copii_rxns, copii_names, copii_systems] =  addPathway('COPII-canonical',copii_rxns,copii_names,copii_systems)       
        for i in range(len(copii_rxns)):
            copii_rxns[i] = copii_rxns[i].replace("!",str(copii_coeff))
            rxns.append(copii_rxns[i])
            rxnNames.append(copii_names[i])
            subSystems.append(copii_systems[i])    
        
        #[rxns,rxnNames] = addPathway('COPII-canonical',rxns,rxnNames)
        connector = 'XXX[g]'

#------------------------------------------------------------------------------
    #Add O-glycosylation reactions
    if PSI[5] != '0': #Has O-glycans?    
        rxns.append(connector + ' -> XXX_preOG[g]')
        rxnNames.append('Start_OG')
        subSystems.append('OG>0')
        number_Oglycans = PSI[5] #Get number of O-Glycans
        OGlyrxns = []
        OGlyrxnNames = []
        OGlysystems = []
        
        [OGlyrxns,OGlyrxnNames,OGlysystems] = addPathwayFromCondition('OG>0',OGlyrxns,OGlyrxnNames,OGlysystems)
        for i in range(len(OGlyrxns)): #Change the '?' for the number of O-glycans
            if '?' in OGlyrxns[i]:
                    if number_Oglycans == '1':
                        OGlyrxns[i] = OGlyrxns[i].replace('? ', '')
                    else:
                        OGlyrxns[i] = OGlyrxns[i].replace('?', number_Oglycans)
        
        for reaction in OGlyrxns:
            rxns.append(reaction)
        for reactionName in OGlyrxnNames:
            rxnNames.append(reactionName)
        for pathwayName in OGlysystems:
            subSystems.append(pathwayName)
        
        connector = 'XXX-Core2[g]'
        
#------------------------------------------------------------------------------
    #Stay in Golgi if protein is localized there
    if PSI[7] == '[g]' or PSI[7] == '[gm]':
        location = PSI[7]
        #rxns.append(connector + ' -> XXX_mature' + location)
        #rxnNames.append('Final_location_' + location)
        if 'SP_degradation' in rxnNames:
            SPaas = count_AAs(sequence[0:22]) #Amino acids in signal peptide assuming length is 22 on average
            rxns[rxnNames.index('SP_degradation')] = substitute_AAs_count(rxns[rxnNames.index('SP_degradation')],SPaas)
        if 'Ubiquitination_degradation' in rxnNames:
            new_aas = count_AAs(sequence[22:])
            rxns[rxnNames.index('Ubiquitination_degradation')] = rxns[rxnNames.index('Ubiquitination_degradation')].replace("!", "?")
            rxns[rxnNames.index('Ubiquitination_degradation')] = substitute_AAs_count(rxns[rxnNames.index('Ubiquitination_degradation')], new_aas)
            rxns[rxnNames.index('Ubiquitination_degradation')] = rxns[rxnNames.index('Ubiquitination_degradation')].replace("?",str(L-22))        
        # Add GPRs
        GPRs = getGPRsFromRxnNames(rxnNames)
        #Change the reaction names and their formulas to include the UniProt Identifier
        for i in range(len(rxns)):
            rxns[i] = insert_prot_name_in_rxnFormula(rxns[i],protName)
        for i in range(len(rxnNames)):
            rxnNames[i] = insert_prot_name_in_rxnName(rxnNames[i],protName)
        rxns.append(protName + location + ' -> ')
        rxnNames.append(protName + '_Final_demand')
        GPRs.append('')
        subSystems.append('Exchange/demand reaction')
        [rxns,rxnNames,GPRs,subSystems] = addCanonicalRxns(rxns,rxnNames,GPRs,subSystems)  
        return rxns,rxnNames,GPRs,subSystems
#------------------------------------------------------------------------------    
    #Add COPI
    elif PSI[7] == '[r]' or PSI[7] == '[rm]':
        rxns.append(connector + ' -> XXX_preCOPI[g]')
        rxnNames.append('Start_COPI')
        subSystems.append('COPI')
        
        copi_rxns = []
        copi_names = []
        copi_systems = []
        [copi_rxns, copi_names, copi_systems] =  addPathway('COPI',copi_rxns,copi_names,copi_systems)       
        for i in range(len(copi_rxns)):
            copi_rxns[i] = copi_rxns[i].replace("!",str(copi_coeff))
            rxns.append(copi_rxns[i])
            rxnNames.append(copi_names[i])
            subSystems.append(copi_systems[i])

        connector = 'XXX_mature[r]'
        location = PSI[7]
        if location == '[r]':
            rxns.append(connector + ' -> ')
            rxnNames.append(protName + '_Final_demand')
            subSystems.append('Exchange/demand reaction')
        elif location == '[rm]':
            rxns.append(connector + ' -> XXX_mature' + location)
            rxnNames.append('Final_location_' + location)
            subSystems.append('Transport, COPI vesicles')
            rxns.append('XXX_mature' + location + ' -> ')
            rxnNames.append(protName + '_Final_demand')
            subSystems.append('Exchange/demand reaction')
        
#------------------------------------------------------------------------------
    #Add Clathrin vesicles
    elif PSI[7]=='[x]' or PSI[7]=='[l]' or PSI[7]=='[d]':
        rxns.append(connector + ' -> XXX-preClathrin[g]')
        rxnNames.append('Start_Clathrin_vesicle')
        subSystems.append('Clathrin vesicles')
        
        clath_rxns = []
        clath_names = []
        clath_systems = []
        [clath_rxns, clath_names, clath_systems] =  addPathway('Clathrin vesicles',clath_rxns,clath_names,clath_systems)       
        for i in range(len(clath_rxns)):
            clath_rxns[i] = clath_rxns[i].replace("!",str(clathrin_coeff))
            rxns.append(clath_rxns[i])
            rxnNames.append(clath_names[i])
            subSystems.append('Clathrin vesicles')
        
        connector = 'XXX_mature[cv]'
        location = PSI[7]
        rxns.append(connector + ' -> XXX_mature' + location)
        rxnNames.append('Final_location_' + location)
        subSystems.append('Transport, Clathrin vesicles')
        rxns.append('XXX_mature' + location + ' -> ')
        rxnNames.append(protName + '_Final_demand')
        subSystems.append('Exchange/demand reaction')
        
#------------------------------------------------------------------------------
    #Send to corresponding location
    else:
        location = PSI[7]
        rxns.append(connector + ' -> XXX-preSV[g]')
        rxnNames.append('Start_Secretion')
        subSystems.append('SV')
        
        sv_rxns = []
        sv_names = []
        sv_systems = []
        [sv_rxns, sv_names, sv_systems] =  addPathway('SV',sv_rxns, sv_names, sv_systems)       
        for i in range(len(sv_rxns)):
            sv_rxns[i] = sv_rxns[i].replace("!",str(clathrin_coeff))
            rxns.append(sv_rxns[i])
            rxnNames.append(sv_names[i])
            subSystems.append(sv_systems[i])      
        
        #[rxns,rxnNames] = addPathway("SV",rxns,rxnNames)
        if location == '':
            location = '[e]'
        rxns.append('XXX_mature[sv]' + ' -> XXX_mature' + location)
        rxnNames.append('Final_location_' + location)
        subSystems.append('Transport, extracellular')
        rxns.append('XXX_mature' + location + ' -> ')
        rxnNames.append('Final_demand')
        subSystems.append('Exchange/demand reaction')
#------------------------------------------------------------------------------
    #Add coeffcients to SP_degradation and Ubiquitin_degradation reactions (if applicable)
    if 'SP_degradation' in rxnNames:
        SPaas = count_AAs(sequence[0:22]) #Amino acids in signal peptide assuming length is 22 on average
        rxns[rxnNames.index('SP_degradation')] = substitute_AAs_count(rxns[rxnNames.index('SP_degradation')],SPaas)
    if 'Ubiquitination_degradation' in rxnNames:
        new_aas = count_AAs(sequence[22:])
        rxns[rxnNames.index('Ubiquitination_degradation')] = rxns[rxnNames.index('Ubiquitination_degradation')].replace("!", "?")
        rxns[rxnNames.index('Ubiquitination_degradation')] = substitute_AAs_count(rxns[rxnNames.index('Ubiquitination_degradation')], new_aas)
        rxns[rxnNames.index('Ubiquitination_degradation')] = rxns[rxnNames.index('Ubiquitination_degradation')].replace("?",str(L-22))
#------------------------------------------------------------------------------
    GPRs = getGPRsFromRxnNames(rxnNames)
#------------------------------------------------------------------------------
    #Change the reaction names and their formulas to include the UniProt Identifier
    for i in range(len(rxns)):
        rxns[i] = insert_prot_name_in_rxnFormula(rxns[i],protName)
    for i in range(len(rxnNames)):
        rxnNames[i] = insert_prot_name_in_rxnName(rxnNames[i],protName)
    [rxns,rxnNames,GPRs,subSystems] = addCanonicalRxns(rxns,rxnNames,GPRs,subSystems)  
    return rxns,rxnNames,GPRs,subSystems

Load recon2.2

In [5]:
model = cobra.io.load_matlab_model('recon2.2.mat')

Create recon2.2s

In [6]:
entryIDs = ["P60568","P01589","P14784","P31785","P05231","P08887","P40189","P13232","P16871","P01579","P01584","P18510",
            "P05112","P05113","P10145","P15248","P22301","P29459","P29460","P35225","P40933","Q96F46","Q9Y2Y9","P09919",
            "P02778","P13500","P10147","P01375","P15692","P01127","P13236","P04141","P09038","P51671","P78556","Q969D9",
           "Q06830","P32119","Q16552","Q96PD4","Q8NAC3","P01009","P02763","P00738","P01011","P02765","P02647","P02787",
           "P02787","P02741"]

# Initialize new model
secretory_model = model.copy()
    
for j in range(len(entryIDs)):
    # Generate lists for reactions (rxns), reaction names (rxnNames), Gene-Protein-Reactions (GPRs), and pathways (subSystems)
    [rxns, rxnNames, GPRs, subSystems] = generateProteinSpecificRxns_A_modified(entryIDs[j])
    # Add new reactions to metabolic model
    for i in range(len(rxns)):
        # Create place holder for creating reactions
        r = secretory_model.reactions[0].copy()
        r.name = rxnNames[i]
        r.id = rxnNames[i]
        r.gene_reaction_rule = GPRs[i]
        r.subsystem = subSystems[i]
        secretory_model.add_reaction(r)
        if r._model is not None:
            r.build_reaction_from_string(reaction_str=rxns[i],rev_arrow='<-',fwd_arrow='->',reversible_arrow='<=>')
    
cobra_io_mat.save_matlab_model(secretory_model, 'recon2.2s_sarscov2.mat')

unknown metabolite 'P60568[c]' created
unknown metabolite 'BAT3[c]' created
unknown metabolite 'CALM[c]' created
unknown metabolite 'P60568-BAT3-CALM[c]' created
unknown metabolite 'ASNA1-atp[c]' created
unknown metabolite 'P60568-ASNA1-atp[c]' created
unknown metabolite 'SEC61C[r]' created
unknown metabolite 'P60568-ASNA1-atp-SEC61C[r]' created
unknown metabolite 'SEC62[r]' created
unknown metabolite 'SEC63[r]' created
unknown metabolite 'TRAM1[r]' created
unknown metabolite 'TRAP[r]' created
unknown metabolite 'DNAJC1[r]' created
unknown metabolite 'SERP1[r]' created
unknown metabolite 'BiP-adp[r]' created
unknown metabolite 'P60568-BiP-adp[r]' created
unknown metabolite 'Translocase[r]' created
unknown metabolite 'ASNA1-adp[c]' created
unknown metabolite 'P60568[r]' created
unknown metabolite 'BiP-atp[r]' created
unknown metabolite 'SRP[c]' created
unknown metabolite 'P60568-SRP[c]' created
unknown metabolite 'SRPR[r]' created
unknown metabolite 'RRBP1[r]' created
unknown metabolite

unknown metabolite 'P01589-G3M9[r]' created
unknown metabolite 'P01589-M9-misfold[r]' created
unknown metabolite 'P01589-M8C-misfold[r]' created
unknown metabolite 'P01589-M7A-misfold[r]' created
unknown metabolite 'P01589-M6-misfold[r]' created
unknown metabolite 'P01589-M5-misfold[r]' created
unknown metabolite 'OS9[r]' created
unknown metabolite 'HSP90B1[r]' created
unknown metabolite 'P01589-M5-unfold-OS9-BiP-adp-HSP90B1[r]' created
unknown metabolite 'P01589-M5-unfold-OS9-SEL1[r]' created
unknown metabolite 'SYVN1[r]' created
unknown metabolite 'P01589-M5-unfold-SEL1-SYVN1[r]' created
unknown metabolite 'UBIQP[c]' created
unknown metabolite 'P01589-M5-unfold-UBIQP-SEL1-SYVN1[r]' created
unknown metabolite 'P01589-M5-unfold-UBIQP[c]' created
unknown metabolite 'SEL1[r]' created
unknown metabolite 'RAD23A[c]' created
unknown metabolite 'P01589-unfold-UBIQP-RAD23A[c]' created
unknown metabolite 'P01589-unfold-Proteasome[c]' created
unknown metabolite 'P01589-G2M9[r]' created
unknown 

Ignoring reaction 'SEC61C' since it already exists.
Ignoring reaction 'TRAP' since it already exists.
Ignoring reaction 'ARF1_gdp_binding' since it already exists.
Ignoring reaction 'ARF1_activation' since it already exists.
Ignoring reaction 'ARF1_gdp_degradation' since it already exists.
Ignoring reaction 'SAR1B_gdp_binding' since it already exists.
Ignoring reaction 'SAR1B_activation' since it already exists.
Ignoring reaction 'SAR1B_gdp_degradation' since it already exists.


unknown metabolite 'P01589-AP-coated[c]' created
unknown metabolite 'P01589_mature[sv]' created
unknown metabolite 'P01589_mature[pm]' created


Ignoring reaction 'RAB1A_gdp_binding' since it already exists.
Ignoring reaction 'RAB6B_gdp_binding' since it already exists.
Ignoring reaction 'RAB8B_gdp_binding' since it already exists.
Ignoring reaction 'RAB1A_gdp_degradation' since it already exists.
Ignoring reaction 'RAB6B_gdp_degradation' since it already exists.
Ignoring reaction 'RAB8B_gdp_degradation' since it already exists.
Ignoring reaction 'RAB1A_activation' since it already exists.
Ignoring reaction 'RAB6B_activation' since it already exists.
Ignoring reaction 'RAB8B_activation' since it already exists.
Ignoring reaction 'BiP_NEF' since it already exists.
Ignoring reaction 'BiP_ATPase' since it already exists.
Ignoring reaction 'BiP_atp_formation' since it already exists.
Ignoring reaction 'BiP_adp_degradation' since it already exists.
Ignoring reaction 'GOLGI_TO_ENDOSOME_3' since it already exists.
Ignoring reaction 'COPI_recycling' since it already exists.
Ignoring reaction 'COPII_recycling' since it already exists.
I

...

Ignoring reaction 'CAATPaser' since it already exists.
Ignoring reaction 'g3m8mpdol_L_to_canonical' since it already exists.
Ignoring reaction 'g3m8mpdol_U_to_canonical' since it already exists.
Ignoring reaction 'doldp_L_to_canonical' since it already exists.
Ignoring reaction 'doldp_U_to_canonical' since it already exists.
Ignoring reaction 'PROTA_formation' since it already exists.
Ignoring reaction 'PROTB_formation' since it already exists.
Ignoring reaction 'H2O2ter' since it already exists.
Ignoring reaction 'P02787_TRANSLATION_protein' since it already exists.
Ignoring reaction 'P02787_post_TRANSLOC_3B_1' since it already exists.
Ignoring reaction 'P02787_post_TRANSLOC_3B_2' since it already exists.
Ignoring reaction 'P02787_post_TRANSLOC_3B_3' since it already exists.
Ignoring reaction 'P02787_co_TRANSLOC_1' since it already exists.
Ignoring reaction 'P02787_co_TRANSLOC_2' since it already exists.
Ignoring reaction 'P02787_co_TRANSLOC_3' since it already exists.
Ignoring reacti

unknown metabolite 'P02741[c]' created
unknown metabolite 'P02741-ASNA1-atp[c]' created
unknown metabolite 'P02741-ASNA1-atp-SEC61C[r]' created
unknown metabolite 'P02741-BiP-adp[r]' created
unknown metabolite 'P02741[r]' created
unknown metabolite 'P02741-SRP[c]' created
unknown metabolite 'P02741-SRP-SRPR-RRBP1[c]' created
unknown metabolite 'P02741-SRP-SRPR-SEC61C[c]' created
unknown metabolite 'P02741-SEC61C[c]' created
unknown metabolite 'P02741-SEC61-SPC[r]' created
unknown metabolite 'P02741-sp[r]' created
unknown metabolite 'P02741_preDSB[r]' created
unknown metabolite 'P02741-ds-P4HB-ox[r]' created
unknown metabolite 'P02741_DSB[r]' created
unknown metabolite 'P02741-LMAN1L[r]' created
unknown metabolite 'P02741-COPII-coated[c]' created
unknown metabolite 'P02741-COPII-uncoated[c]' created
unknown metabolite 'P02741[g]' created
unknown metabolite 'P02741-preSV[g]' created
unknown metabolite 'P02741-AP-coated[c]' created
unknown metabolite 'P02741_mature[sv]' created


Ignoring reaction 'SEC61C' since it already exists.
Ignoring reaction 'TRAP' since it already exists.
Ignoring reaction 'ARF1_gdp_binding' since it already exists.
Ignoring reaction 'ARF1_activation' since it already exists.
Ignoring reaction 'ARF1_gdp_degradation' since it already exists.
Ignoring reaction 'SAR1B_gdp_binding' since it already exists.
Ignoring reaction 'SAR1B_activation' since it already exists.
Ignoring reaction 'SAR1B_gdp_degradation' since it already exists.
Ignoring reaction 'RAB1A_gdp_binding' since it already exists.
Ignoring reaction 'RAB6B_gdp_binding' since it already exists.
Ignoring reaction 'RAB8B_gdp_binding' since it already exists.
Ignoring reaction 'RAB1A_gdp_degradation' since it already exists.
Ignoring reaction 'RAB6B_gdp_degradation' since it already exists.
Ignoring reaction 'RAB8B_gdp_degradation' since it already exists.
Ignoring reaction 'RAB1A_activation' since it already exists.
Ignoring reaction 'RAB6B_activation' since it already exists.
Ign

unknown metabolite 'P02741_mature[e]' created


Ignoring reaction 'EXOCYST_degradation' since it already exists.
Ignoring reaction 'SECRETION_3' since it already exists.
Ignoring reaction 'SRP' since it already exists.
Ignoring reaction 'SRPR' since it already exists.
Ignoring reaction 'SPC' since it already exists.
Ignoring reaction 'SPC_degradation' since it already exists.
Ignoring reaction 'co_TRANSLOC_7' since it already exists.
Ignoring reaction 'TRAP_degradation' since it already exists.
Ignoring reaction 'OST_complex' since it already exists.
Ignoring reaction 'Proteasome_degradation' since it already exists.
Ignoring reaction 'CAATPaser' since it already exists.
Ignoring reaction 'g3m8mpdol_L_to_canonical' since it already exists.
Ignoring reaction 'g3m8mpdol_U_to_canonical' since it already exists.
Ignoring reaction 'doldp_L_to_canonical' since it already exists.
Ignoring reaction 'doldp_U_to_canonical' since it already exists.
Ignoring reaction 'PROTA_formation' since it already exists.
Ignoring reaction 'PROTB_formation'